In [3]:
import sys, os
sys.path.append('./venv/lib/site-packages')

In [4]:
from PIL import Image 
import sys 
from pdf2image import convert_from_path, convert_from_bytes

from base64 import b64decode
import codecs

import easyocr
import io
import numpy as np

In [155]:
base64String = "..."

In [156]:
def pil_to_cv2(image):
    open_cv_image = np.array(image)
    return open_cv_image[:, :, ::-1].copy()

In [157]:
bytesStr = b64decode(base64String, validate=True)

if bytesStr[0:4] != b'%PDF':
    raise ValueError('Missing the PDF file signature')


pages = convert_from_bytes(bytesStr, fmt='jpeg', poppler_path=r'.\venv\poppler\bin')

reader = easyocr.Reader(["es"]) # need to run only once to load model into memory
results=[]
for page in pages:
    result=reader.readtext(pil_to_cv2(page),detail=0)
    for i in range(len(result)):
        results.append(result[i])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [158]:
print(results)

['REPUBLICA DE COLOMBIA', 'IDENTIFICACION PERSONAL', 'CEDULA DE CIUDADANIA', '21.163.421', 'NUMERO', 'ROCHA DE PIÑEROS', 'APELLIDOS', 'CLARA INES', 'NOMBRES', 'Jhe Pelede Pees', 'IRMA', 'FECHA DE NACIMIENTO', '11-AGO-1944', 'BOGOTA D.C', '(CUNDINAMARCA)', 'LUGAR DE NACIMIENTO', '1.62', 'A+', 'F', 'ESTATURA', 'G.S', 'RH', 'SEXO', '30-SEP-1965 ZIPAQUIRA', 'FECHA Y LUGAR DE EXPEDICION', 'nM', 'Um', 'REGISTRADOH NACIGNAL', 'INDICE DERECHO', 'JuaM caalos Galindo vatha', '4-1500117-45160205-F-0021163421-20070515', '0030107135H 01', '993893666']


In [159]:
text = open("textoCedula.txt", "w")


In [160]:
#Para la cara frontal de la 
for i in range(len(results)):
    if results[i] == "NUMERO" or results[i] == "NÚMERO":

        if results[i+3] == "APELLIDOS" and results[i+4] != "NOMBRES":
            text.write("Nombre: " + str(results[i+4]) + "\nApellidos: " + str(results[i+2]) + "\nNúmero: " + str(results[i+1])  + "\n")
            break
        elif results[i+2] == "APELLIDOS" and results[i+3] != "NOMBRES" and results[i-2] == "CEDULA DE CIUDADANIA" or results[i-2] == "CÉDULA DE CIUDADANIA":
            text.write("Nombre: " + str(results[i+3]) + "\nApellidos: " + str(results[i+1]) + "\nNúmero: " + str(results[i-1])  + "\n")
            break
    
    elif results[i] == "APELLIDOS":
        if results[i-3] == "CEDULA DE CIUDADANIA" or results[i-3] == "CÉDULA DE CIUDADANIA" and results[i+1] != "NOMBRES":
            text.write("Nombre: " + str(results[i+1]) + "\nApellidos: " + str(results[i-1]) + "\nNúmero: " + str(results[i-2])  + "\n")
            break  
        else:
            text.write("La cara frontal de la cedula no es clara. Por favor subir otro archivo")
            break
    
    
        



    



In [161]:
#Para la reversa de la cedula
for i in range(len(results)):
    if results[i] == "FECHA DE NACIMIENTO":
        text.write("Fecha de nacimiento: " + str(results[i+1]) + "\nLugar de nacimiento: " + str(results[i+2]) + "\n")
    
    if results[i] == "ESTATURA":
        if results[i-1] == "M" or results[i-1] == "F":
            text.write("Estatura: " + str(results[i-3]) + "\nG.S. RH: " + str(results[i-2]) + "\nSexo: " + str(results[i-1]) + "\n")
        else:
            text.write("Estatura: " + str(results[i-2]) + "\nG.S. RH: " + str(results[i-1]) + "\n")

       
        




In [162]:
text.close()